In [ ]:
from syft_client import SyftboxManager
from pathlib import Path
import requests
import syft_job as sj

In [ ]:
ds_manager, do_manager = SyftboxManager.pair_with_in_memory_connection(
        use_in_memory_cache=False
    )

## Datasets

### Download

In [ ]:
MOCK_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/mock/sales.csv"
PRIVATE_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/private/sales.csv"


In [ ]:
# Download Mock Dataset
mock_local_path = Path("./sales_mock.csv")
r = requests.get(MOCK_DATASET_URL)
with open(mock_local_path, "wb") as f:
    f.write(r.content)

# Download Private Dataset
private_local_path = Path("./sales_private.csv")
r = requests.get(PRIVATE_DATASET_URL)
with open(private_local_path, "wb") as f:
    f.write(r.content)

In [ ]:
# Create a example readme.md file
readme_path = Path("./readme.md")
with open(readme_path, "w") as f:
    f.write("# Dataset Readme\nThis is a readme file for the dataset.")

### Create Dataset

In [ ]:
do_manager.create_dataset(
    name="my dataset",
    mock_path=mock_local_path,
    private_path=private_local_path,
    summary="This is a summary",
    readme_path=readme_path,
    tags=["tag1", "tag2"],
)



In [ ]:
do_manager.datasets.get_all()

In [ ]:
ds_manager.datasets.get_all()

In [ ]:
do_manager.sync()
ds_manager.sync()

In [ ]:
ds_manager.datasets.get_all()

### Submit Job

In [ ]:
ds_job_client = sj.get_client(ds_manager.syftbox_folder, ds_manager.email)

In [ ]:
ds_job_client.jobs

In [ ]:
code_file_name = "main.py"

In [ ]:
do_manager.syftbox_folder

In [ ]:
# %%writefile {code_file_name}

# import syft_client as sc

# data_path = "syft://private/sales-dataset/sales.csv"
# resolved_path = sc.resolve_path(data_path)

# import pandas as pd

# df = pd.read_csv(resolved_path)

# print(f"Total Sales: {(df['quantity'] * df['price_per_unit']).sum()}")

In [ ]:
%%writefile {code_file_name}

print("Hello World")

In [ ]:
ds_job_client.submit_python_job(
    user=do_manager.email,
    code_path=code_file_name
)

In [ ]:
ds_job_client.jobs

In [ ]:
ds_manager.sync()
do_manager.sync()

### Review Jobs

In [ ]:
do_job_client = sj.get_client(do_manager.syftbox_folder, do_manager.email)

In [ ]:
do_job_client.jobs

In [ ]:
do_job_client.jobs[-1].approve()

### View Results

In [ ]:
ds_job_client.jobs[-1].stdout